In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

**2. Data Understanding using SparkSQL**

In [8]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-03 13:34:59--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   191MB/s    in 3.5s    

2022-02-03 13:35:03 (188 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [9]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [10]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [11]:
raw_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('2008.csv')

In [12]:
raw_df

DataFrame[Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [13]:
raw_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [14]:
raw_df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [15]:
raw_df.describe('DepTime','TaxiOut','TaxiIn','DayOfWeek','Distance','ArrDelay').show()

+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|summary|           DepTime|           TaxiOut|           TaxiIn|         DayOfWeek|         Distance|         ArrDelay|
+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|           7009728|           7009728|          7009728|           7009728|          7009728|          7009728|
|   mean|1333.8300461105448|16.453045177492882|6.860851704974527|3.9241815088973495|726.3870294253928| 8.16845238729114|
| stddev|478.06889486629836|11.332798654232155|4.933649371300466|1.9882589459851212|562.1018034840403|38.50193694882867|
|    min|                 1|                 0|                0|                 1|              100|               -1|
|    max|                NA|                NA|               NA|                 7|              999|               NA|
+-------+------------------+----

In [16]:
raw_df.groupBy('month').count().sort('count',ascending=False).show()

+-----+------+
|month| count|
+-----+------+
|    7|627931|
|    3|616090|
|    8|612279|
|    6|608665|
|    5|606293|
|    1|605765|
|    4|598126|
|    2|569236|
|   10|556205|
|   12|544958|
|    9|540908|
|   11|523272|
+-----+------+



In [17]:
raw_df.groupBy('month').count().sort('count',ascending=False).withColumnRenamed("count","numMonth").show()

+-----+--------+
|month|numMonth|
+-----+--------+
|    7|  627931|
|    3|  616090|
|    8|  612279|
|    6|  608665|
|    5|  606293|
|    1|  605765|
|    4|  598126|
|    2|  569236|
|   10|  556205|
|   12|  544958|
|    9|  540908|
|   11|  523272|
+-----+--------+



In [18]:
raw_df.withColumn('ArrDelay', raw_df['ArrDelay'].cast('int')).groupBy('month').avg('ArrDelay').show()

+-----+------------------+
|month|     avg(ArrDelay)|
+-----+------------------+
|    7| 9.975049681276131|
|   11| 2.015857969430839|
|    3| 11.19236458018227|
|    8|  6.91091468997087|
|    5| 5.978448290248828|
|    6|13.266756009659792|
|    9|0.6977328787273043|
|    1|10.188855960349496|
|   10|0.4154954706912698|
|    4| 6.807297481094145|
|   12|16.680505081496417|
|    2|13.077836997760205|
+-----+------------------+



In [19]:
raw_df.groupBy('month').agg({'month':'count', 'ArrDelay':'avg'}).show()
raw_df.groupBy('month').agg({'month':'count', 'ArrDelay':'avg'}).printSchema()

+-----+------------+------------------+
|month|count(month)|     avg(ArrDelay)|
+-----+------------+------------------+
|    7|      627931| 9.975049681276131|
|   11|      523272| 2.015857969430839|
|    3|      616090| 11.19236458018227|
|    8|      612279|  6.91091468997087|
|    5|      606293| 5.978448290248828|
|    6|      608665|13.266756009659792|
|    9|      540908|0.6977328787273043|
|    1|      605765|10.188855960349496|
|   10|      556205|0.4154954706912698|
|    4|      598126| 6.807297481094145|
|   12|      544958|16.680505081496417|
|    2|      569236|13.077836997760205|
+-----+------------+------------------+

root
 |-- month: string (nullable = true)
 |-- count(month): long (nullable = false)
 |-- avg(ArrDelay): double (nullable = true)



Discovered from Stack Overflow!

In [20]:
raw_df.groupBy('month').agg({'month':'count', 'ArrDelay':'avg'}).sort('count(month)',ascending=False).withColumnRenamed("count(month)","numMonth").show()

+-----+--------+------------------+
|month|numMonth|     avg(ArrDelay)|
+-----+--------+------------------+
|    7|  627931| 9.975049681276131|
|    3|  616090| 11.19236458018227|
|    8|  612279|  6.91091468997087|
|    6|  608665|13.266756009659792|
|    5|  606293| 5.978448290248828|
|    1|  605765|10.188855960349496|
|    4|  598126| 6.807297481094145|
|    2|  569236|13.077836997760205|
|   10|  556205|0.4154954706912698|
|   12|  544958|16.680505081496417|
|    9|  540908|0.6977328787273043|
|   11|  523272| 2.015857969430839|
+-----+--------+------------------+



In [23]:
from pyspark.sql.functions import *

In [25]:
raw_df.select('DepTime').show()

+-------+
|DepTime|
+-------+
|   2003|
|    754|
|    628|
|    926|
|   1829|
|   1940|
|   1937|
|   1039|
|    617|
|   1620|
|    706|
|   1644|
|   1426|
|    715|
|   1702|
|   1029|
|   1452|
|    754|
|   1323|
|   1416|
+-------+
only showing top 20 rows



In [26]:
raw_df.withColumn('lengthDepTime', length('DepTime')).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|lengthDepTime|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-------------+
|2008|    1|         3|        4|   2003|      1955|

In [50]:
raw_df\
.withColumn('DepTime', when(length('DepTime')<4,format_string('0%s', 'DepTime'))\
            .otherwise(raw_df.DepTime))\
            .show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [51]:
raw_df\
.withColumn('DepTime', when(length('DepTime')<4,format_string('0%s', 'DepTime'))\
            .otherwise(raw_df.DepTime))\
            .withColumn('DepTime', to_timestamp('DepTime','HHmm'))\
            .show()

+----+-----+----------+---------+-------------------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|            DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------------------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|1970-01-01 20:03:00|     

In [52]:
raw_df\
.withColumn('DepTime', when(length('DepTime')<4,format_string('0%s', 'DepTime'))\
            .otherwise(raw_df.DepTime))\
            .withColumn('DepTime', to_timestamp('DepTime','HHmm'))\
            .groupBy('DepTime').count().orderBy('DepTime')\
            .show()

+-------------------+------+
|            DepTime| count|
+-------------------+------+
|               null|140582|
|1970-01-01 01:00:00|   644|
|1970-01-01 01:01:00|   527|
|1970-01-01 01:02:00|   511|
|1970-01-01 01:03:00|   461|
|1970-01-01 01:04:00|   469|
|1970-01-01 01:05:00|   595|
|1970-01-01 01:06:00|   465|
|1970-01-01 01:07:00|   461|
|1970-01-01 01:08:00|   487|
|1970-01-01 01:09:00|   439|
|1970-01-01 01:10:00|   161|
|1970-01-01 01:11:00|   116|
|1970-01-01 01:12:00|    95|
|1970-01-01 01:13:00|   113|
|1970-01-01 01:14:00|   127|
|1970-01-01 01:15:00|   146|
|1970-01-01 01:16:00|   103|
|1970-01-01 01:17:00|    87|
|1970-01-01 01:18:00|   108|
+-------------------+------+
only showing top 20 rows



In [59]:
raw_df\
.withColumn('DepTime', when(length('DepTime')==3,format_string('0%s', 'DepTime'))\
            .when(length('DepTime')==2,format_string('00%s', 'DepTime'))\
            .when(length('DepTime')==1,format_string('000%s', 'DepTime'))\
            .when(length('DepTime')==0,format_string('0000%s', 'DepTime'))\
            .otherwise(raw_df.DepTime))\
            .withColumn('DepTime', to_timestamp('DepTime','HHmm'))\
            .groupBy('DepTime').count().orderBy('DepTime')\
            .show()

+-------------------+------+
|            DepTime| count|
+-------------------+------+
|               null|136767|
|1970-01-01 00:01:00|   566|
|1970-01-01 00:02:00|   428|
|1970-01-01 00:03:00|   429|
|1970-01-01 00:04:00|   420|
|1970-01-01 00:05:00|   460|
|1970-01-01 00:06:00|   397|
|1970-01-01 00:07:00|   392|
|1970-01-01 00:08:00|   357|
|1970-01-01 00:09:00|   366|
|1970-01-01 00:10:00|   427|
|1970-01-01 00:11:00|   344|
|1970-01-01 00:12:00|   343|
|1970-01-01 00:13:00|   314|
|1970-01-01 00:14:00|   322|
|1970-01-01 00:15:00|   425|
|1970-01-01 00:16:00|   330|
|1970-01-01 00:17:00|   327|
|1970-01-01 00:18:00|   347|
|1970-01-01 00:19:00|   289|
+-------------------+------+
only showing top 20 rows



In [60]:
raw_df\
.withColumn('DepTime', timeperiod(to_timestamp('DepTime','HHmm')))\
.groupBy('DepTime').count().orderBy('DepTime').show()

NameError: ignored

หาไม่เจอว่า syntax ของ timeperiod ทำงานยังไงครับ